## Simulation

We can present the same information as a simulation of the motion of the bell.

In [1]:
%matplotlib inline
from scipy.special import ellipk
import scipy.integrate as integrate
import numpy as np
from numpy import sqrt, sin, cos, log, pi, floor
from functools import partial
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, Layout, ToggleButton
import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
g = 9.81
phi1 = 30*pi/180
phi0 = -30*pi/180

def deriv(l_b, y, t):
    [theta, vtheta] = y
    theta_dot  = vtheta
    vtheta_dot = - (g / l_b) * sin(theta)
    return np.array([theta_dot, vtheta_dot])

# Exact formula from elliptic integral
def exact_period(l_b, theta_0):
    return 4 * sqrt(l_b / g) * ellipk(sin(theta_0/2)**2)

def rk4(f, x, t, step):
    k1 = step * f(x, t)
    x1 = x + 0.5*k1
    k2 = step * f(x1, t+0.5*step)
    x2 = x + 0.5*k2
    k3 = step * f(x2, t+0.5*step)
    x3 = x + k3;
    k4 = step * f(x3, t+step)
    return (k1 + 2*k2 + 2*k3 + k4) / 6.0

def integrate_rk4(func, y0, tindex):
    y = [np.array(y0)]
    t_last = tindex[0]
    for t in tindex[1:]:
        step = t - t_last
        y.append(y[-1] + rk4(func, y[-1], t, step))
        t_last = t
    return np.array(y)

In [3]:
%%capture
# set up the plot

timestep = 50 # timestep in milliseconds
num_timesteps = 0
bell_l_b = 0.6
t = 0
y = [0,0]

fig3 = plt.figure(figsize=(6,6), dpi=80)
fig3_ax = fig3.add_subplot(111, autoscale_on=False, xlim=(-1.5,1.5), ylim=(-1.5,1.5))
fig3_ax.set_xticklabels([])
fig3_ax.set_yticklabels([])
line, = fig3_ax.plot([], [], 'o-', lw=2)  # solid line with circular markers
time_template = 'time = %.1fs' # format as a float to the tenths place
time_text = fig3_ax.text(0.05, 0.9, '', transform=fig3_ax.transAxes)

In [4]:

def next_frame(i):
    global y, t, bell_l_b, line, time_text
    next_y = rk4(partial(deriv, bell_l_b), y, t, timestep/1000)
    [y[0],y[1]] = [y[0]+next_y[0],y[1]+next_y[1]]
    t = t + timestep/1000
    line.set_data([0,sin(y[0])], [0,-cos(y[0])])
    time_text.set_text(time_template % t)
    return (line,)

def setup_plot(theta_0, l_b):
    global y, t, bell_l_b, num_timesteps, anim
    bell_l_b = l_b
    y = [theta_0 * pi/180, 0]
    t = 0
    Period = exact_period(l_b, theta_0*pi/180)
    num_timesteps = int(floor(1000.0 * Period / timestep))

    
    # Call the animator. Only re-draw the parts that have changed (via blit=True)
    anim = animation.FuncAnimation(fig3, next_frame, \
                                   frames=num_timesteps, \
                                   interval=timestep, \
                                   blit=True)

# animation with initial angle and l_b
setup_plot(179, 0.6)
HTML(anim.to_html5_video())

In [5]:
def deriv_with_clapper(l_b, l_c, c, alpha, y, t):
    [theta, vtheta, phi, vphi] = y
    theta_dot  = vtheta
    vtheta_dot = - (g / l_b) * np.sin(theta)

    phi_dot = vphi

    # note sign error in the original notes 
    vphi_dot = -vtheta_dot * (1 + (c / l_c) * np.cos(phi - alpha)) \
               + (c / l_c) * (vtheta**2) * np.sin(phi - alpha) - (g / l_c) * np.sin(theta+phi)
    return np.array([theta_dot, vtheta_dot, phi_dot, vphi_dot])

In [6]:
def integrate_rk4_clapper(y0, t, step, l_b, l_c, c, alpha, clapper):
    y = y0 + rk4(partial(deriv_with_clapper, l_b, l_c, c, alpha), y0, t, step)
    # check for clapper clapping the bell
    if y[2] > phi1:
        clapper = True
        y[2] = phi1
        y[3] = 0
    elif y[2] < phi0:
        clapper = True
        y[2] = phi0
        y[3] = 0
    else:
        clapper = False
    return [clapper, y]

In [30]:
%%capture
# set up the plot

timestep = 50 # timestep in milliseconds

fig_bc = plt.figure(figsize=(6,6), dpi=80)
fig_bc_ax = fig_bc.add_subplot(111, \
                            autoscale_on=False, xlim=(-1.5,1.5), ylim=(-1.5,1.5))
fig_bc_ax.set_xticklabels([])
fig_bc_ax.set_yticklabels([])
bell_line, = fig_bc_ax.plot([], [], 'o-', lw=2)  # solid line with circular markers
bl0, = fig_bc_ax.plot([], [], '-', lw=1)
bl1, = fig_bc_ax.plot([], [], '-', lw=1)
clapper_line, = fig_bc_ax.plot([], [], 'o-', lw=2)  # solid line with circular markers
time_template = 'time = %.1fs' # format as a float to the tenths place
time_text = fig3_ax.text(0.05, 0.9, '', transform=fig3_ax.transAxes)

In [32]:
def bc_next_frame(i):
    global y, t, bell_l_b, bell_l_c, bell_c, bell_alpha, bell_clapper, line, time_text
    [next_clapper, y] = integrate_rk4_clapper(y, t, timestep/1000, \
                                                   bell_l_b, bell_l_c, bell_c, bell_alpha, bell_clapper)
    t = t + timestep/1000
    bell_line.set_data([0,sin(y[0])], [0,-cos(y[0])])
    bl0.set_data([0,sin(y[0]+phi0)], [0,-cos(y[0]+phi0)])
    bl1.set_data([0,sin(y[0]+phi1)], [0,-cos(y[0]+phi1)])
    clapper_line.set_data([0,0.5*sin(y[0]+y[2])], [0,-0.5*cos(y[0]+y[2])])
    time_text.set_text(time_template % t)
    return (line,)

def bc_setup_plot(theta_0, l_b, l_c, c, alpha):
    global y, t, bell_l_b, bell_l_c, bell_c, bell_alpha, bell_clapper, num_timesteps, anim
    bell_l_b = l_b
    bell_l_c = l_c
    bell_c = c
    bell_alpha = alpha
    bell_clapper = True
    y = np.array([theta_0 * pi/180, 0, phi1, 0])
    t = 0
    Period = exact_period(l_b, theta_0*pi/180)
    num_timesteps = int(floor(1000.0 * Period / timestep))

    
    # Call the animator. Only re-draw the parts that have changed (via blit=True)
    anim = animation.FuncAnimation(fig_bc, bc_next_frame, \
                                   frames=num_timesteps, \
                                   interval=timestep, \
                                   blit=True)

# animation with initial angle and l_b
bc_setup_plot(179, 0.6, 1.0, 0.1, 0)
HTML(anim.to_html5_video())